<a href="https://colab.research.google.com/github/vux979/Medical-sentiment-analysis/blob/main/colab_%E6%95%B4%E4%BD%93%E8%BF%90%E8%A1%8C%EF%BC%88%E5%B7%B2%E6%9B%B4%E6%94%B9%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Mar 14 14:55:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    56W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 下载并处理数据集

## 数据下载

[kaggle数据集下载参考网址](https://segmentfault.com/a/1190000022134947)

In [2]:
!pip install kaggle

In [3]:
import json
token = {"username":"shuxizhang","key":"bb59cfc7fa45f101dcf52ee15438874d"}
# 我的室友就用我的号吧，如果是其他的访问者还是改一下token
with open('/content/kaggle.json', 'w') as file:
  json.dump(token, file)

In [4]:
!mkdir -p ~/.kaggle

In [5]:
!cp /content/kaggle.json ~/.kaggle/

In [6]:
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle config set -n ./kaggle-dataset -v /yelp

- ./kaggle-dataset is now set to: /yelp


In [8]:
!kaggle datasets download -d yelp-dataset/yelp-dataset

yelp-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
!ls

Dataset_User_Agreement.pdf	     yelp_academic_dataset_checkin.json
kaggle.json			     yelp_academic_dataset_review.json
sample_data			     yelp_academic_dataset_tip.json
transformers			     yelp_academic_dataset_user.json
yelp_academic_dataset_business.json  yelp-dataset.zip


In [ ]:
!unzip yelp-dataset.zip

Archive:  yelp-dataset.zip
replace Dataset_User_Agreement.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!ls

## 处理数据

In [ ]:
import json
import os
from tqdm import tqdm
import gc
import re
import pandas as pd

In [ ]:
s = "./yelp_academic_dataset_business.json"

data_business = []

with open(s, 'r', encoding='UTF-8') as load_f:
    for line in tqdm(load_f):
        data_business.append(json.loads(line))

In [ ]:
# 商店信息的数量
len(data_business)

In [ ]:
# 查看商店信息的存储格式
data_business[0]

In [ ]:
# 找到 categories 中包括 str1 & str2 这两个类的商店
str1 = 'Health & Medical'
str2 = 'Doctors'

business_hmd = []
for i in tqdm(range(len(data_business))):
    if data_business[i]["categories"] != None: # categories 不为空值
        flag = data_business[i]["categories"].rfind(str1) # 找不到 str1 就返回 -1
        if flag != -1:
            flag = data_business[i]["categories"].rfind(str2) # 找不到 str2 就返回 -1
            if flag != -1:
                business_hmd.append(data_business[i])

In [ ]:
len(business_hmd)

In [ ]:
business_hmd[0]

In [ ]:
del(data_business) # 清内存，删除列表
gc.collect()

### 抽取商店id

In [ ]:
business_hmd_id = []
for element in tqdm(business_hmd):
    business_hmd_id.append(element["business_id"])
len(business_hmd_id)

In [ ]:
del(business_hmd) # 清内存
gc.collect()

### 读取评论

In [ ]:
s = "./yelp_academic_dataset_review.json"
reviews_hmd = []
with open(s, 'r', encoding='UTF-8') as load_f:
  for line in tqdm(load_f):
    line_dict = json.loads(line)
    if line_dict['business_id'] in business_hmd_id:
      reviews_hmd.append(line_dict)

In [ ]:
len(reviews_hmd)

### 提取正负样本

In [ ]:
pos = []
neg = []
mid = []

In [ ]:
def cleanReview(subject):
    newSubject = re.sub(r'\n', '', subject)
    newSubject = re.sub(r"\'", "'", newSubject)
    return newSubject

In [ ]:
for element in tqdm(reviews_hmd):
    if element['stars'] >= 4.0:
        pos.append(cleanReview(element['text']))
    elif element['stars'] <=2.0:
        neg.append(cleanReview(element['text']))
    else:
        mid.append(cleanReview(element['text']))

In [ ]:
print('pos: ', len(pos), '\n', 'neg: ', len(neg), '\n', 'mid: ', len(mid))

In [ ]:
df_pos = pd.DataFrame(columns=['']) # 创建dataframe， 定义一个没有名字的空列
df_neg = pd.DataFrame(columns=[''])
df_mid = pd.DataFrame(columns=[''])

df_pos[''] = pos # 给空列赋值
df_neg[''] = neg
df_mid[''] = mid

# 训练

In [ ]:
!pip3 install torch torchvision
!pip install transformers

In [ ]:
import os
from tqdm import tqdm
import random
import time
import torch
from torch import nn
import torch.utils.data as Data
import  torch.nn.functional as F
import sys
import numpy as np
import pandas as pd 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.__version__, device)

In [ ]:
def loadfile():
    combined=np.concatenate((pos[:30000], neg[:30000]))
    # print(type(pos[0][0])) <str>
    y = np.concatenate((np.ones(len(pos[:30000]),dtype=int), np.zeros(len(neg[:30000]),dtype=int)))
    # pos 1, neg 0

    return combined, y

In [ ]:
#构造数据
def data_classfier():
    combined,y = loadfile()
    data = combined
    labels = y
    print('Shape of data tensor:', len(data))
    print('Shape of label tensor:', len(labels))
    


    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)  #打乱
    data = data[indices]

    labels = labels[indices]
    
    VALIDATION_SPLIT = 0.5
    nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
    x_train = data[:-nb_validation_samples]
    y_train = labels[:-nb_validation_samples]
    x_val = data[-nb_validation_samples:]
    y_val = labels[-nb_validation_samples:]
    return x_train,y_train,x_val,y_val

In [ ]:
x_train,y_train,x_val,y_val = data_classfier()

In [ ]:
print(type(x_train))
print(type(y_train))
print(type(x_val))
print(type(y_val))
print(len(x_train))
print(len(y_train))
print(len(x_val))
print(len(y_val))

## 导入bert

In [ ]:
from transformers import BertModel, BertTokenizer
# 这里我们调用bert-base模型，同时模型的词典经过小写处理
model_name = 'bert-base-uncased'
# 读取模型对应的tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name, cache_dir='./transformers/bert-base-uncased/')
# 载入模型
model = BertModel.from_pretrained(model_name, cache_dir='./transformers/bert-base-uncased/')

In [ ]:
model.to(device)
model.eval()

In [ ]:
train_data = []
for i in  range(len(x_train)):
    train_data.append([])
    train_data[i].append(x_train[i])
    train_data[i].append(y_train[i])

In [ ]:
test_data = []
for i in  range(len(x_val)):
    test_data.append([])
    test_data[i].append(x_val[i])
    test_data[i].append(y_val[i])

In [ ]:
def pretreatment(original_data):
    i = 0
    for element in tqdm(original_data):
        temporary = []
        original_data[i][0] = torch.tensor(tokenizer.encode(element[0], add_special_tokens=True, max_length = 512, padding='max_length', truncation=True))
        temporary.append(element[1])
        original_data[i][1] = torch.tensor(temporary)
        i = i+1
    features =  torch.cat([original_data[i][0].unsqueeze(0).long() for i in range(len(original_data))])
    labels =  torch.cat( [original_data[i][1].long() for i in range(len(original_data))], 0)
    return features, labels

In [ ]:
train_set = Data.TensorDataset(*(pretreatment(train_data)))
test_set = Data.TensorDataset(*(pretreatment(test_data)))

In [ ]:
batch_size = 8
train_iter = Data.DataLoader(train_set, batch_size)# , shuffle=True)
test_iter = Data.DataLoader(test_set, batch_size)# , shuffle=True)

## 模型定义

In [ ]:
class GlobalMaxPool1d(nn.Module):
    def __init__(self):
        super(GlobalMaxPool1d, self).__init__()
    def forward(self, x):
         # x shape: (batch_size, channel, seq_len)
        return F.max_pool1d(x, kernel_size=x.shape[2]) # shape: (batch_size, channel, 1)

In [ ]:
class TextCNN(nn.Module):
    def __init__(self, embed_size, kernel_sizes, num_channels):
        super(TextCNN, self).__init__()
        # self.embedding = nn.Embedding(len(vocab), embed_size)
        # 不参与训练的嵌入层
        # self.constant_embedding = nn.Embedding(len(vocab), embed_size)
        self.dropout = nn.Dropout(0.5)
        self.decoder_1 = nn.Linear(sum(num_channels), 256)
        self.decoder_2 = nn.Linear(256, 2)
        # 时序最大池化层没有权重，所以可以共用一个实例
        self.pool = GlobalMaxPool1d()
        self.convs = nn.ModuleList()  # 创建多个一维卷积层
        
        for c, k in zip(num_channels, kernel_sizes):
            self.convs.append(nn.Conv1d(in_channels = embed_size, 
                                        out_channels = c, 
                                        kernel_size = k))

    def forward(self, inputs):
        outputs = model(inputs)[0] #shape（512， 768）
        embeddings = outputs # (batch, seq_len, embed_size)
        # 根据Conv1D要求的输入格式，将词向量维，即一维卷积层的通道维(即词向量那一维)，变换到前一维
        embeddings = embeddings.permute(0, 2, 1) # 交换维度的函数
        # 对于每个一维卷积层，在时序最大池化后会得到一个形状为(批量大小, 通道大小, 1)的
        # Tensor。使用flatten函数去掉最后一维，然后在通道维上连结
        encoding = torch.cat([self.pool(F.relu(conv(embeddings))).squeeze(-1) for conv in self.convs], dim=1)
        # 应用丢弃法后使用全连接层得到输出
        middle = self.decoder_1(self.dropout(encoding))
        
        outputs = self.decoder_2(self.dropout(F.relu(middle)))
        return outputs

In [ ]:
embed_size, kernel_sizes, nums_channels = 768, [3, 4, 5], [100, 100, 100]
net = TextCNN(embed_size, kernel_sizes, nums_channels)

In [ ]:
def evaluate_accuracy(data_iter, net, device=None):
    if device is None and isinstance(net, torch.nn.Module):
        # 如果没指定device就使用net的device
        device = list(net.parameters())[0].device 
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in tqdm(data_iter, desc ='evaluate'):
            if isinstance(net, torch.nn.Module):
                net.eval() # 评估模式, 这会关闭dropout
                acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
                net.train() # 改回训练模式
            else: # 自定义的模型, 3.13节之后不会用到, 不考虑GPU
                if('is_training' in net.__code__.co_varnames): # 如果有is_training这个参数
                    # 将is_training设置成False
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() 
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
            n += y.shape[0]
    return acc_sum / n

In [ ]:
def train(train_iter, test_iter, net, loss, optimizer, device, num_epochs):
    net = net.to(device)
    print("training on ", device)
    batch_count = 0
    best_acc = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        print(time.strftime("%Y-%m-%d \t %H:%M:%S", time.localtime()), end='\t')
        for X, y in tqdm(train_iter, desc ='train'):
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y) 
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        if test_acc > best_acc:
            best_acc = test_acc
            print("saving", end='\t')
            PATH = "./BertBaseEpoch1_yelp_" + str(best_acc) + ".pth"
            torch.save(net, PATH) 
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, test_acc, time.time() - start))

In [ ]:
lr, num_epochs = 0.001, 10
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)
loss = nn.CrossEntropyLoss()
train(train_iter, test_iter, net, loss, optimizer, device, num_epochs)

# 模型测试

In [ ]:
def predict_sentiment(net, sentence):
    """sentence是词语的列表"""
    device = list(net.parameters())[0].device
    sentence = torch.tensor(tokenizer.encode(s, add_special_tokens=True, max_length = 512, padding='max_length', truncation=True), device=device)
    result = net(sentence.view((1, -1)))
    m = nn.Softmax(dim=1)
    print(m(result))
    label = torch.argmax(result, dim=1)
    return 'positive' if label.item() == 1 else 'negative'

In [ ]:
print("请输入一句与医疗相关的英文：")
s = input() 
print(predict_sentiment(net, s))